# Chapter4: 셀레니움 실전 테크닉

- 여러태그 동시 제어
  - 여러 태그 찾기 및 클릭하기
- 동적 대기 기법
  - 암시적 대기
  - 명시적 대기
- 고급 입력 컨트롤
  - 입력 값 삭제
  - 순차적 키 입력
- 셀렉트박스 조작 방법
  - Select 클래스를 이용한 드롭다운 메뉴 선택
- 여러 페이지 관리
  - 페이지 안에 다른 페이지가 있는 경우(iframe 제어방법)
  - 새로운 창 제어 방법

## 여러 태그 동시 제어

예제 사이트 => "https://startcoding.pythonanywhere.com/basic"  
예제 사이트에서 label 즉, filter 부분을 모두 click하는 예시이다.

만약 `로그인`상태에서 이렇게 클릭을 빠르게 하는 동작을 하면 문제가 발발하므로 속도를 줄이자.

> 💡 label[for="something"] # label tag 중 for 속성이 something인 태그 가져오기


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

url = "https://startcoding.pythonanywhere.com/basic"
driver = webdriver.Chrome()
driver.get(url)

labels = driver.find_elements(By.CSS_SELECTOR, "label[for]")

In [7]:
import time

for label in labels: 
  label.click()
  time.sleep(0.5)
  

In [10]:
for label in labels:
  print(label.text, label.get_attribute("for"))

노트북 (10) category-1
데스크탑 (10) category-2
휴대폰 (10) category-3
모니터 (5) category-4
카메라 (128) category-5
악세사리 (740) category-6
SAMSUNG (578) brand-1
LG (125) brand-2
HP (755) brand-3
DELL (578) brand-4
ASUS (125) brand-5
SONY (755) brand-6


## 동적 대기 기법

웹 페이지의 태그가 `즉시 로드가 되지 않고 어떤 동작이나 시간이 지난 후에 나타는 경우`, 예를 들면 javascript에 의해 동적으로 추가되

이러한 경우를 처리하기 위해 `selenium`에서는 두 가지 대기 기법(암시적 대기, 명시적 대기)을 제공한다.

### 암시적 대기(Implicit Wait)
암시적 대기는 `설정한 시간`동안 요소를 대기하는 방법이다.  
- `설정한 시간 안에 찾을 시`: 다음 코드 진행
- `설정한 시간 안에 못 찾을 시`: `NoSuchElementException` 예외 발생


> 💡 암시적 대기는 `웹 드라이버 객체에 대해 한 번 설정하면, 객체가 살아있는 동안 계속 유효`하다.
> 또한, 매우 중요한 부분인데 `요소를 찾는 행위`에만 기다린다. 즉 암시적 대기의 적용 범위는 `탐색(find)`에만 적용되며 `클릭,입력, 가시성 등`에는 적용되는 범주가 아니다.  
> 따라서, 위에서 말한 `객체가 살아있는 동안 계속 유효`함은 `이후 모든 요소를 탐색하는 코드`에 대해서 전역적으로 적용된다는 의미이다.

```python
driver = webdriver.Chrome()
driver.implicitly_wait(10) # 10초 동안 대기
```

### 명시적 대기(Explicit Wait)

특정 시간 동안 기다리는 것은 암묵적 대기와 같지만 `특정 조건이 만족할 때까지 대기`하는 기법이다. `WebDriverWait`과 `expected_conditions`를 이용하여 조건을 확인하고 대기할 수 있다.  

조건 예시
1. 특정 태그가 나타났을 때
2. 태그가 클릭 가능해질 때

조건 부분만 암시적 대기와 차이가 나며 시간 안에 조건이 만족하지 않으면 예외가 발생한다.
- `설정한 시간 안에 조건이 만족될 시`: 다음 코드 진행
- `설정한 시간 안에 조건이 만족되지 않을 시`: `TimeoutException` 예외 발생


```python
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.get("https://www.naver.com")

# 최대 10초 동안 대기
# #query1인 태그가 나타날 때까지 대기하며 이 때 `display:none`과 상관없이 나타나기만 하면 조건이 True가 된다.
element = WebDriverWait(driver, 10).until(
  EC.presence_of_element_located((By.CSS_SELECTOR, "#query1"))
)
```

### 명시적 대기 VS 암시적 대기
- 범위
  - 암시적 대기: 웹 드라이버 `객체 전체`에 대해 설정, 모든 태그를 찾을 때 적용
  - 명시적 대기: `특정 태그나 조건`에 대해 설정된다.
- 유연성
  - 암시적 대기: 그냥 기다리는 것만 함.
  - 명시적 대기: 다양한 조건(clickable, visible, presence)등에 대해 대기할 수 있어 더욱 유연하다.
- 예외처리방식
  - 암시적 대기: `NoSuchElementException` 예외 발생
  - 명시적 대기: `TimeoutException` 예외 발생




### 암시적 대기 예시

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome()

driver.get("https://www.naver.com")



찾는 요소가 있을 때 -> 바로 대기 취소

In [ ]:
driver.implicitly_wait(5)
search = driver.find_element(By.CSS_SELECTOR, "#query")

찾는 요소가 없을 때 -> 기다렸다가 예외 발생

In [27]:
driver.implicitly_wait(5)
search = driver.find_element(By.CSS_SELECTOR, "#query1")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#query1"}
  (Session info: chrome=141.0.7390.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
0   chromedriver                        0x0000000100677cf8 cxxbridge1$str$ptr + 2895872
1   chromedriver                        0x000000010066fc34 cxxbridge1$str$ptr + 2862908
2   chromedriver                        0x0000000100195570 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74324
3   chromedriver                        0x00000001001dcf34 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 367640
4   chromedriver                        0x000000010021e3d8 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 635068
5   chromedriver                        0x00000001001d10f8 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_

### 명시적 대기 예시

In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome()

driver.get("https://www.naver.com")


정해진 시간 안에 조건을 만족할 때

In [33]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

search = WebDriverWait(driver, 10).until(
  EC.presence_of_element_located((By.CSS_SELECTOR, "#query"))
)
search.send_keys("셀레니움")

정해진 시간 안에 조건을 만족하지 않을 때

In [34]:
search = WebDriverWait(driver, 3).until(
  EC.presence_of_element_located((By.CSS_SELECTOR, "#query1")) # 👉🏻 없는 id
)
search.send_keys("셀레니움")

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x000000010133fcf8 cxxbridge1$str$ptr + 2895872
1   chromedriver                        0x0000000101337c34 cxxbridge1$str$ptr + 2862908
2   chromedriver                        0x0000000100e5d570 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74324
3   chromedriver                        0x0000000100ea4f34 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 367640
4   chromedriver                        0x0000000100ee63d8 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 635068
5   chromedriver                        0x0000000100e990f8 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 318940
6   chromedriver                        0x000000010130381c cxxbridge1$str$ptr + 2648868
7   chromedriver                        0x0000000101306df8 cxxbridge1$str$ptr + 2662656
8   chromedriver                        0x00000001012e4334 cxxbridge1$str

In [35]:
driver.quit()